In [ ]:
from dotenv import load_dotenv
load_dotenv()
import requests
import json
import os
from pprint import pprint
from utilities import *
import pandas as pd
import pygsheets

api_key = os.environ.get('riot_api_key')
my_puuid = os.environ.get('my_puuid')
test_match = os.environ.get('test_match')
rank1_puuid = os.environ.get('rank1_puuid')
json_dir = "JSONs"
service_account = pygsheets.authorize(service_account_file='JSONs/spreadsheetautomator-455618-bd558fe6d57a.json')

with open(os.path.join(json_dir, "archetype_items.json"), "r") as file:
    archetype_items = json.load(file)

with open(os.path.join(json_dir, "champ_classes.json"), "r") as file:
    classes = json.load(file)

with open(os.path.join(json_dir, "champ_items.json"), "r") as file:
    champ_items = json.load(file)

with open(os.path.join(json_dir, "champ_positions.json"), "r") as file:
    positions = json.load(file)

with open(os.path.join(json_dir, "most_recent.json"), "r") as file:
    recent = json.load(file)

with open(os.path.join(json_dir, "item_to_id.json"), "r") as file:
    item_to_id = json.load(file)

with open(os.path.join(json_dir, "perk_to_id.json"), "r") as file:
    perk_to_id = json.load(file)

with open(os.path.join(json_dir, "other_to_id.json"), "r") as file:
    other_to_id = json.load(file)

item_to_id = {int(k): v for k, v in item_to_id.items()}
perk_to_id = {int(k): v for k, v in perk_to_id.items()}
other_to_id = {int(k): v for k, v in other_to_id.items()}

In [5]:
history = get_match_history('na1', rank1_puuid, 80)

rank1_df = pd.DataFrame()
for match in history[:1]:
    thisgame = parse_match_data('na1', match, rank1_puuid)
    rank1_df = pd.concat([rank1_df, thisgame])
pd.set_option("display.max_columns", None)

rank1_df

,match_id,game_creation,game_duration,game_start,game_end,kills,deaths,assists,gold_earned,gold_spent,champ_level,neutral_minions_killed,total_minions_killed,turret_kills,summoner1_id,summoner2_id,damage_dealt_to_objectives,damage_dealt_to_turrets,dragon_kills,item0,item1,item2,item3,item4,item5,item6,items_purchased,champion_name,champion_transform,bounty_level,killing_sprees,first_blood_assist,first_blood_kill,first_tower_assist,first_tower_kill,double_kills,total_enemy_jungle_minions_killed,largest_killing_spree,total_damage_dealt,total_damage_dealt_to_champions,total_damage_shielded_on_teammates,total_damage_taken,total_heal,total_heals_on_teammates,all_in_pings,assist_me_pings,enemy_missing_pings,enemy_vision_pings,hold_pings,get_back_pings,need_vision_pings,on_my_way_pings,push_pings,command_pings,detector_wards_placed,vision_score,vision_wards_bought_in_game,wards_killed,game_ended_in_early_surrender,game_ended_in_surrender,participant_id,puuid,riot_id_game_name,riot_id_tagline,team_id,win,defense,flex,offense,keystone,primary_1,primary_2,primary_3,secondary_1,secondary_2,atakhan,baron,champion,dragon,grubs,inhibitor,tower,rift_herald
0,NA1_5257762669,1743557411960,1772,1743557430795,1743559203021,5,4,3,11821,12000,14,1,222,0,4,1,10775,3260,0,Essence Reaver,Infinity Edge,Cloak of Agility,Rapid Firecannon,Last Whisper,Slightly Magical Footwear,Farsight Alteration,24,Lucian,0,5,2,False,False,False,False,2,0,3,176231,36605,0,25513,2849,0,0,0,12,30,0,13,3,29,1,21,4,31,5,5,False,False,9,EOtvtWaRDJYIrY4W3YoiJN3KCxU5eZor5RJZGdCUMhbtL8...,Alpha Smoker,420,200,False,Health,Adaptive Force,Attack Speed,Electrocute,Sudden Impact,Grisly Mementos,Ultimate Hunter,Biscuit Delivery,Magical Footwear,0,0,22,2,2,0,3,1


In [ ]:
# pd.set_option('display.max_rows', None)
ladder = get_high_elo_players()

In [8]:
summonerspells = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/summoner-spells.json'

res = requests.get(summonerspells).json()
ids = extract_json(res, 'id')
name = extract_json(res, 'name')

sums_dict = dict(map(lambda i, j: (i, j), ids, name))
sums_dict

with open(os.path.join(json_dir, "other_to_id.json"), 'w') as file:
    json.dump(sums_dict, file, indent=4)
          